In [ ]:
import pandas as pd
from functools import reduce
import pickle
import numpy as np
#pandas 1.4.4
#python version
#

In [ ]:
file_path = "C:/Users/DanielleDuncan/Pictures/Camera Roll/Masters-Thesis/Word Databases/"
result = pd.read_pickle(file_path + "1--Complete Word List")
words = result["feature"].tolist()

**Getting the Coefficients out of Log Reg**

In [ ]:
open_path = "C:/Users/DanielleDuncan/Desktop/THESIS/logreg results/"

In [ ]:
model_year = '2010'

with open(open_path + model_year +'_results.pkl', 'rb') as f:
    results = pickle.load(f)

vocabu = results['tfidf'].vocabulary_

word_vector = np.zeros(len(vocabu))
for word in words:
    if word in vocabu:
        index = vocabu[word]
        word_vector[index] = 1

coefficients = results['model'].coef_[0]
word_coefficients = {word: coefficient for word, coefficient in zip(vocabu.keys(), coefficients)}

coef_store = []

for word in words:
    if word in word_coefficients:
        coef_store.append((f"{word}, {word_coefficients[word]}"))

results_coef = pd.DataFrame(coef_store)
results_coef = pd.concat([results_coef[[0]], results_coef[0].str.split(', ', expand=True)], axis=1)
results_coef.set_axis(['drop', 'feature', model_year + '_coef'], axis='columns', inplace=True)
merged_left = pd.merge(left=result, right=results_coef, how='left', left_on='feature', right_on='feature')
merged_left = merged_left.drop(['drop'], axis=1)

df = merged_left
df = df.drop_duplicates()


In [ ]:
year_list = ['2011', '2012', '2013', '2014', '2015',"2016", "2017", "2018", "2019", "2020", "2021", "2022"]


In [ ]:
for year in year_list:

    with open(open_path + year +'_results.pkl', 'rb') as f:
        results = pickle.load(f)

    vocabu = results['tfidf'].vocabulary_

    word_vector = np.zeros(len(vocabu))
    for word in words:
        if word in vocabu:
            index = vocabu[word]
            word_vector[index] = 1

    coefficients = results['model'].coef_[0]
    word_coefficients = {word: coefficient for word, coefficient in zip(vocabu.keys(), coefficients)}

    coef_store = []

    for word in words:
        if word in word_coefficients:
            coef_store.append((f"{word}, {word_coefficients[word]}"))

    results_coef = pd.DataFrame(coef_store)
    results_coef = pd.concat([results_coef[[0]], results_coef[0].str.split(', ', expand=True)], axis=1)
    results_coef.set_axis(['drop', 'feature', year + '_coef'], axis='columns', inplace=True)
    merged_left = pd.merge(left=result, right=results_coef, how='left', left_on='feature', right_on='feature')
    merged_left = merged_left.drop(['drop'], axis=1)

    df = pd.merge(left = df, right = merged_left, how= 'inner', left_on=['feature','source'], right_on=['feature', 'source'])
    df = df.drop_duplicates()

In [ ]:
df.to_csv("LogReg CSV PAC Yearly Results")

In [ ]:
file_name = "LogReg--Pickle PAC Yearly Results"

with open(file_name, 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)